In [1]:
import json
import requests
import pandas as pd

from tqdm import tqdm

import chromadb
from sentence_transformers import SentenceTransformer

/home/marco/Desktop/Coding/refBro/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = SentenceTransformer("sentence-transformers/allenai-specter")

In [3]:
with open('data/dynamical_systems.json', 'r') as file:
    papers_db = json.load(file)

papers_db = [paper for paper in papers_db if paper["title"] is not None]

In [4]:
sep = model.tokenizer.sep_token
# concat title and abstract as done by the authors https://github.com/allenai/specter
db_tabs_concat = [d["title"] + sep + d["abstract"] for d in papers_db]

In [5]:
embeddings = model.encode(db_tabs_concat)

KeyboardInterrupt: 

In [ ]:
# Initialize ChromaDB client and collection
chroma_client = chromadb.PersistentClient(path="database/dynamical_systems")
collection = chroma_client.create_collection(name="papers")

for i, (paper, embedding) in tqdm(enumerate(zip(papers_db, embeddings))):
    collection.add(
        documents=[paper["title"] + sep + paper["abstract"]],
        embeddings=[embedding.tolist()],
        metadatas=[{"title": paper["title"], "abstract": paper["abstract"]}],
        ids=[str(i)]
    )
